In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib import gridspec
from scipy.optimize import curve_fit

# input file and data

In [ ]:
#file paths and names
stockpath = './UT/Dt-Count/'
stockname = '15e-1A_EachIntervals'

filenames = []
storepaths = []

num_filled = 5

for i in range(10):
    filenames.append("./rawdata/UT_test/")
    storepaths.append("./UT/Dt_Count/")
    pass

filenames[0] += "15e-1A_04e-1SecInterval_100times.txt"
storepaths[0] += ""

filenames[1] += "15e-1A_08e-1SecInterval_100times.txt"
storepaths[1] += ""

filenames[2] += "15e-1A_12e-1SecInterval_100times.txt"
storepaths[2] += ""

filenames[3] += "15e-1A_16e-1SecInterval_100times.txt"
storepaths[3] += ""

filenames[4] += "15e-1A_20e-1SecInterval_100times.txt"
storepaths[4] += ""

filenames[5] += ""
storepaths[5] += ""

filenames[6] += ""
storepaths[6] += ""

filenames[7] += ""
storepaths[7] += ""

filenames[8] += ""
storepaths[8] += ""

filenames[9] += ""
storepaths[9] += ""

In [ ]:
#read files
raws = []

error = []
direction = []
overflow = []
timer_count = []
refcount = []

for i in range(num_filled):
    raws.append(0)
    
    error.append(np.zeros(sum([1 for _ in open(filenames[i])])))
    direction.append(np.zeros(sum([1 for _ in open(filenames[i])])))
    overflow.append(np.zeros(sum([1 for _ in open(filenames[i])])))
    timer_count.append(np.zeros(sum([1 for _ in open(filenames[i])])))
    refcount.append(np.zeros(sum([1 for _ in open(filenames[i])])))
    
    pass

for i in range(num_filled):
    error[i], direction[i], overflow[i], timer_count[i], refcount[i] = np.loadtxt(filenames[i], comments="#",unpack=True)
    raws[i] = len(error[i])
    
    pass

raws

## set variables
* Please do not switch the for loops

In [ ]:
operating_interval = 0.4
Deg = 360/52000

time = []

for i in range(num_filled):
    time.append(timer_count[i]/200e6)

In [ ]:
skip = 100

In [ ]:
#modifing and cutting
single_refcount = []

diffReference = []
modTime = []
edge_point = []
stable_place = []
flattened_count = []
diffFlat = []

OpeTime = []
single_diffFlat = []

for i in range(num_filled):
    single_refcount.append(np.zeros(len(refcount[i])))
    pass

for i in range(num_filled):
    count_offset = 0
    casp_flag = 0
    casp = np.where(np.diff(refcount[i]) <= -100)[0]
    
    for j in range(len(refcount[i])):
        single_refcount[i][j] = refcount[i][j]+count_offset
        if j == casp[casp_flag]:
            count_offset += refcount[i][casp[casp_flag]]
            if len(casp) > casp_flag+1:
                casp_flag += 1
                pass
            pass
        pass
    
    diffReference.append(np.diff(single_refcount[i][::skip]))
    modTime.append(time[i][::skip])
    edge_point.append([])
    
    for k in range(len(diffReference[i])):
        if diffReference[i][k] > 0.:
            if diffReference[i][k-1] <= 0.:
                edge_point[i].append(k-10)
                pass
            pass
        pass
    
    edge_point[i].append(max(edge_point[i])+max(np.diff(edge_point[i])))
    
    stable_place.append([])
    for l in range(len(edge_point[i])):
        stable_place[i] = np.append(stable_place[i], np.where(time[i] == modTime[i][edge_point[i][l]])[0])
        pass
    
    flattened_count.append([])
    for m in range(len(stable_place[i])):
        flattened_count[i] = np.append(flattened_count[i], single_refcount[i][round(stable_place[i][m])])
    
    diffFlat.append([])
    diffFlat[i] = np.append(diffFlat[i], np.diff(flattened_count[i]))
    
    for j in range(len(diffFlat[i])):
        OpeTime = np.append(OpeTime, (i+1)*operating_interval)
        pass
    
    single_diffFlat = np.append(single_diffFlat,diffFlat[i])
    
    pass

## check some points

In [ ]:
file_num = 0

for i in range(len(edge_point[file_num])-1):
    plt.plot(modTime[file_num][edge_point[file_num][i]:edge_point[file_num][i+1]]-modTime[file_num][edge_point[file_num][i]],
            diffReference[file_num][edge_point[file_num][i]:edge_point[file_num][i+1]])
    pass
plt.ylim([-50,1000])

# Plots

In [ ]:
fig0, axarr0 = plt.subplots(2,3,figsize=(16,12))
ax0 = axarr0.ravel()

for i in range(num_filled):
    ax0[i].hist(diffFlat[i]*Deg)
    ax0[i].set_xlabel('difference of degree')
    ax0[i].set_ylabel('number of events')
    ax0[i].set_title(f'operating for {round((i+1)*operating_interval,2)} sec')

In [ ]:
fig0.savefig(stockpath + stockname + '_HistDiffDeg.png')

In [ ]:
fig1 = plt.figure(figsize=(10,7))
plt.scatter(OpeTime, single_diffFlat*Deg,s=5)
plt.title('Dt-Degree')
plt.xlabel('operating time')
plt.ylabel('difference of degree')

In [ ]:
fig1.savefig(stockpath + stockname + '_DtDeg.png')

In [ ]:
fig2, axarr2 = plt.subplots(2,3,figsize=(16,12))
ax2 = axarr2.ravel()

for i in range(num_filled):
    ax2[i].scatter(flattened_count[i][:-1]%52000*Deg,diffFlat[i]*Deg)
    ax2[i].set_xlabel('position from reference')
    ax2[i].set_ylabel('proceeded degree')
    ax2[i].set_title(f'operating for {round((i+1)*operating_interval,2)} sec')

In [ ]:
fig2.savefig(stockpath + stockname + '_PositionDeg.png')

In [ ]:
fig3 = plt.figure(figsize=(10,7))

for i in range(num_filled):
    plt.clf()
    for j in range(len(edge_point[i])-1):
        plt.plot(modTime[i][edge_point[i][j]:edge_point[i][j+1]]-modTime[i][edge_point[i][j]],
                diffReference[i][edge_point[i][j]:edge_point[i][j+1]]*Deg)
        pass
    plt.xlabel('cycle time [sec]')
    plt.ylabel('degree gradient')
    
    fig3.savefig(stockpath + stockname + f'_CycleGrad{i}.png')
    pass

# Others